In [61]:
#%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
#%load_ext autotime

import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib notebook
notebook_rc = dict(mpl.rcParams)
# mpl.rcParams.update(notebook_rc)

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import cufflinks as cf

from numpy.lib.stride_tricks import as_strided

from mypy.theutility import *
from mypy.attributes import attribute_accessor as AA
from mypy.object import Namespace as Ob
from mypy.quickvis import canvas
#canvas(12.3, 12.3)

from scipy.spatial.distance import pdist, cdist, squareform
import scipy.linalg as linalg
from time import time

from pathlib import Path
data_dir = Path('./cinc2011/')

# Signal Quality of ECG sensors

In [2]:
# cool list
# good: 
# bad: 132
def flatten_dimensions(a, lo, hi):
   return a.reshape(a.shape[:lo] + (prod(a.shape[lo:hi+1]),) + a.shape[hi+1:])

In [3]:
train_dir = data_dir / 'set-a'
idents_list = []
for suffix in '', '-acceptable', '-unacceptable':
    _name = 'RECORDS' + suffix
    with (train_dir / _name).open('r') as f:
        idents_list.append([int(x) for x in f.readlines()])
[ident_all, ident_good, ident_bad] = [sp.asarray(idents) for idents in idents_list]
orig_tables = {}
for i in ident_all:
    orig_tables[i] = pd.read_csv(train_dir / '{}.txt'.format(i), header=None, usecols=range(1, 13))
    orig_tables[i].columns = 'I II III aVR aVL aVF V1 V2 V3 V4 V5 V6'.split()

### The 12-lead ECG

|<img src="https://upload.wikimedia.org/wikipedia/commons/c/c9/Limb_leads.svg" width="300" />|<img src="https://upload.wikimedia.org/wikipedia/commons/4/41/Precordial_leads_in_ECG.png" width="200" />|
|---|---|
|<img src="https://upload.wikimedia.org/wikipedia/commons/1/19/Limb_leads_of_EKG.png" width="300" />|<img src="https://upload.wikimedia.org/wikipedia/commons/0/0e/EKG_leads.png" width="200" />|

## Data

[**PhysioNet/Computing in Cardiology Challenge 2011**](https://physionet.org/challenge/2011/)

#### Training Set A
1000 counts of 10 second long 12-lead ECG records.  
Acceptable/unacceptable quality labels available

#### Training Set B
500 counts of 10 second long 12-lead ECG records.  
Quality labels not publicly available.  
** *not used**

In [4]:
def _normalize(table):
    table_min, table_max = table.min(), table.max()
    table_range = table_max - table_min
    return (table - table_min) / (sp.where(table_range == 0, 1, table_range))
norm_tables = dictmap(_normalize, orig_tables)
def _downsample(table):
    s = (table.index.to_series() / 5).astype(table.index.dtype)
    return table.groupby(s).mean()
downsamp_tables = dictmap(_downsample, norm_tables)

In [92]:
thetables = norm_tables
def add_subplot(fig, df, row, col):
    for column in df.columns:
        fig.append_trace({'x': df.index, 'y': df[column], 'name': column}, row, col)

#### Normalized signals

In [117]:
fig = tls.make_subplots(rows=1, cols=2, subplot_titles=('Good', 'Bad'), shared_yaxes=True)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



In [118]:
add_subplot(fig, norm_tables[ident_good[43]] + list(range(11, -1, -1)), 1, 1)
add_subplot(fig, norm_tables[ident_bad[132]] + list(range(11, -1, -1)), 1, 2)
py.iplot(fig)

The draw time for this plot will be slow for all clients.


/opt/anaconda3/lib/python3.5/site-packages/plotly/plotly/plotly.py:1443: UserWarning:

Estimated Draw Time Too Long



#### Downsampled signals

In [119]:
fig = tls.make_subplots(rows=1, cols=2, subplot_titles=('Good', 'Bad'), shared_yaxes=True)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



In [120]:
add_subplot(fig, downsamp_tables[ident_good[43]] + list(range(11, -1, -1)), 1, 1)
add_subplot(fig, downsamp_tables[ident_bad[132]] + list(range(11, -1, -1)), 1, 2)
py.iplot(fig)

In [5]:
def _variation(table):
    return table.diff().abs().sum()
variation_tables = dictmap(_variation, norm_tables)
variations_good = sp.stack(variation_tables[i].as_matrix() for i in ident_good)
variations_bad = sp.stack(variation_tables[i].as_matrix() for i in ident_bad)

In [126]:
_v = [sp.log(v.min(axis=1) + 1) for v in (variations_good, variations_bad)]
data = [go.Histogram(x=_v[0], name='good', nbinsx=30, opacity=0.55),
        go.Histogram(x=_v[1], name='bad', nbinsx=30, opacity=0.55)]
layout = go.Layout(title="Histogram of variations of ECG records",  xaxis=dict(title='variation in record'), barmode='overlay')
py.iplot(go.Figure(data=data, layout=layout))

In [122]:
ident_good_missing = ident_good[sp.where((variations_good == 0).sum(axis=1) == 1)[0]]
print('IDs of good records with a flat signal (0 variation) in one lead: ', ident_good_missing)
ident_good_complete = ident_good[sp.where((variations_good == 0).sum(axis=1) == 0)[0]]
print('The number of good records with no flat signals: ', len(ident_good_complete))
ident_bad_complete = ident_bad[sp.where((variations_bad == 0).sum(axis=1) == 0)[0]]
print('The number of bad records with no flat signals: ', len(ident_bad_complete))
ident_all_complete = sp.concatenate([ident_good_complete, ident_bad_complete])

IDs of good records with a flat signal (0 variation) in one lead:  [1968453 2080991 2151032 2536401 2537839 2883516]
The number of good records with no flat signals:  767
The number of bad records with no flat signals:  96


## Algorithms

### References
**Dimensionality reduction: A comparative review**  
LJP Van der Maaten, EO Postma, HJ Van den Herik  
(2009) Technical Report TiCC TR 2009-005

**Common manifold learning using alternating-diffusion**  
RR Lederman, R Talmon  
(2014) Yale University, New Haven, CT, USA, Tech. Rep. YALEU/DCS/TR-1497

**Out-of-sample extensions for lle, isomap, mds, eigenmaps, and spectral clustering**  
Y Bengio, JF Paiement, P Vincent, O Delalleau, N Le Roux, M Ouimet  
(2004) Advances in neural information processing systems 16, 177-184

## Goal
Create a Signal Quality Index for short window intervals of ECG signals

### Method
- Keep only records where all signals are not flat
- Partition data into training set and validation set (50/50, evenly splitting good and bad)
- Sample 0.8-second long windows from 431 training records (15 samples from each) $\Longrightarrow X\supset X_0$
- Nonlinearly reduce dimensionality of samples  (expect ECGs to follow a low dimensional pattern)
 - Perform custom alternating Laplacian eigenmap $\Longrightarrow X_0\stackrel{\text{LEM}}\to Y_0$
- Use extension algorithm to describe how to map unseen data in the same nonlinear way $\Longrightarrow X\stackrel{\text{LEM}'}\to Y$
- Take another sample batch from the training set and train a classifier after reducing dimensions
  $\Longrightarrow X_1\stackrel{\text{LEM}'}\to Y_1\stackrel{\text{train}}{\leftrightarrow}\text{labels}$
- Test the pipeline with samples from the validation set $\Longrightarrow X_{\text{test}}\stackrel{\text{LEM}'}\to Y_{\text{test}}\stackrel{\text{eval}}{\leftrightarrow}\text{labels}$

In [7]:
theident_all = ident_all_complete
theident_good = ident_good_complete
theident_bad = ident_bad_complete

from sklearn.model_selection import StratifiedShuffleSplit
skf = StratifiedShuffleSplit(n_splits=1, test_size=0.5)
_label = sp.in1d(theident_all, theident_bad)
_train, _validation = skf.split(theident_all, _label).__next__()
ident_train = theident_all[_train]
label_train = _label[_train]
ident_validation = theident_all[_validation]
label_validation = _label[_validation]

In [81]:
def rolling_window(x, window, axis):
    if axis < 0:
        axis += len(x.shape)
    shape = x.shape[:axis] + (x.shape[axis] - window + 1, window) + x.shape[axis + 1:]
    strides = x.strides[:axis] + (x.strides[axis],) + x.strides[axis:]
    return as_strided(x, shape=shape, strides=strides)
thetables = downsamp_tables
n_samples = 20
sample_length = 80
def sample_record(i):
    matrix = thetables[i].as_matrix()
    samples_idx = sp.random.permutation(matrix.shape[0] - sample_length + 1)[:n_samples]
    windows = rolling_window(matrix, sample_length, 0)
    return windows[samples_idx], samples_idx
def create_samples(ident):
    sr = [sample_record(i) for i in ident]
    sample_windows = sp.stack([a[0] for a in sr])
    sample_indices = sp.stack([a[1] for a in sr])
    return sample_windows, sample_indices

In [156]:
symscale = lambda M, x: x[:, sp.newaxis] * M * x
alpha = .5
nnn = 60

t0 = time()
sample_windows0, sample_indices0 = create_samples(ident_train)
XX0 = flatten_dimensions(sample_windows0, 0, 1)
X_0 = flatten_dimensions(XX0, 1, 2)
l0 = sp.concatenate([lbl * sp.ones(n_samples) for lbl in label_train])

W0 = sp.zeros((len(XX0), len(XX0)))
sigma0_alt = [None for i in range(12)]
d0_alt = [None for i in range(12)]
for lead in range(12):
    _X = XX0[..., lead]
    _E = squareform(pdist(_X, 'sqeuclidean'))
    sigma0_alt[lead] = sp.sqrt(sp.sort(_E, axis=0)[nnn])
    _W = sp.exp(-symscale(_E, 1 / (sigma0_alt[lead] + 0.00001)))
    _d = _W.sum(axis=1)
    # alpha normalization
    _W = symscale(_W, _d ** -alpha)
    # make transition matrix
    d0_alt[lead] = _W.sum(axis=1)
    W0 += (1 / d0_alt[lead])[:, sp.newaxis] * _W
d0 = W0.sum(axis=0)
A0 = (1 / d0)[:, sp.newaxis] * W0

# find eigenvectors of diffusion transition matrix
mu, U = sp.linalg.eig(A0)
# eigh puts eigenvalues in ascending order, but we want descending order
order = sp.argsort(-mu)
mu = mu[order]
U = U.T[order].T
ev = sp.real(mu[:200])
Y0 = sp.real(U[:, :200])

t1 = time()
t1 - t0

452.5711669921875

In [80]:
def save():
    sp.savez(
        "samp40nnn800x.npz",
        sigma0_alt=sigma0_alt,
        d0_alt=d0_alt,
        mu=mu,
        ev=ev,
        Y0=Y0,
        Y=Y,
        Y_test=Y_test,
        sample_windows0 = sample_windows0,
        sample_indices0 = sample_windows0,
        sample_windows = sample_windows,
        sample_indices = sample_windows,
        sample_windows_test = sample_windows_test,
        sample_indices_test = sample_windows_test
    )
#save()

### Visualization via Laplacian Eigenmap

In [163]:
data = [go.Scatter3d(x=Y0[:, 4], y=Y0[:, 2], z=Y0[:, 3], mode='markers', 
                     marker=dict(size='2', color=label_train * 1, showscale=True, colorscale='Jet', opacity=.8 ))]
layout = go.Layout(title="Laplacian Eigenmap for X0")
py.iplot(go.Figure(data=data, layout=layout))

In [157]:
t0 = time()
n_samples = 20
sample_windows, sample_indices = create_samples(ident_train)
XX = flatten_dimensions(sample_windows, 0, 1)
X_ = flatten_dimensions(XX, 1, 2)
l = sp.concatenate([lbl * sp.ones(n_samples) for lbl in label_train])

def y(XX):
    W = sp.zeros((len(XX), len(XX0)))
    for lead in range(12):
        _X0 = XX0[..., lead]
        _X = XX[..., lead]
        _E = cdist(_X, _X0, 'sqeuclidean')
        _sigma = sp.sqrt(sp.sort(_E, axis=1)[:, nnn])
        _W = sp.exp(-(1 / (_sigma + 0.00001))[:, sp.newaxis] * _E * (1 / (sigma0_alt[lead] + 0.00001)))
        _d = _W.sum(axis=1)
        _W = (_d ** -alpha)[:, sp.newaxis] * _W * (d0_alt[lead] ** -alpha)
        _d = _W.sum(axis=1)
        W += (1 / _d)[:, sp.newaxis] * _W
    d = W.sum(axis=1)
    A = (1 / d)[:, sp.newaxis] * W
    return sp.real(A @ Y0 / ev)
Y = y(XX)

t1 = time()
t1 - t0

151.77474808692932

In [148]:
fig = tls.make_subplots(rows=1, cols=2, subplot_titles=('X0', 'X1'))

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [164]:
data = [go.Scatter3d(x=Y[:, 1], y=Y[:, 2], z=Y[:, 3], mode='markers', 
                     marker=dict(size='2', color=label_train * 1, showscale=True, colorscale='Jet', opacity=.8 ))]
layout = go.Layout(title="Laplacian Eigenmap for X1")
py.iplot(go.Figure(data=data, layout=layout))

In [165]:
pd.DataFrame(ev).iplot(title="Eigenvalues")

In [158]:
n_samples = 20
sample_windows_test, sample_indices_test = create_samples(ident_validation)
XX_test = flatten_dimensions(sample_windows_test, 0, 1)
X_test = flatten_dimensions(XX_test, 1, 2)
l_test = sp.concatenate([lbl * sp.ones(n_samples) for lbl in label_validation])
Y_test = y(XX_test)

In [170]:
class ClassifierEvaluation():
    def __init__(self, classifier, fit, predict, y_transform=identity):
        self.classifier = classifier
        self.fit_method = fit
        self.predict_method = predict
        self.y_transform = y_transform
    def fit(self, X, y):
        return self.fit_method(self.classifier)(X, y)
    def test(self, X, y):
        self.predicted = self.y_transform(self.predict_method(self.classifier)(X))
        info = (self.predicted >= 0.5) - y
        self.error = (info == -1).sum(), (info == 1).sum()
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
xgb = ClassifierEvaluation(XGBClassifier(), AA.fit, AA.predict)
logistic = ClassifierEvaluation(LogisticRegression(), AA.fit, AA.decision_function, lambda y: 1 / (1 + sp.exp(-y)))
knn = ClassifierEvaluation(KNeighborsClassifier(n_neighbors=3), AA.fit, AA.predict)
models = dict(xgb=xgb, logistic=logistic, knn=knn)

In [175]:
def test_lem(model_names):
    for name in model_names:
        models[name].fit(Y, l)
        models[name].test(Y_test, l_test)
def test_raw(model_names):
    for name in model_names:
        models[name].fit(sp.concatenate([X_0, X_]), sp.concatenate([l0, l]))
        models[name].test(X_test, l_test)

## Results

Not very good actually...

Running the classifiers even without dimension reduction actually arguably do better

In [176]:
/print len(ident_validation)*n_samples sum(label_validation==0)*n_samples sum(label_validation==1)*n_samples
test_raw(['logistic'])
/print "raw" models['logistic'].error
test_lem(['logistic'])
/print "lem" models['logistic'].error
# kNN tends to be very bad

8640 7680 960
raw (827, 121)
lem (960, 0)


In [88]:
/print len(ident_validation)*n_samples sum(label_validation==0)*n_samples sum(label_validation==1)*n_samples
test_raw()
/print "raw" models['xgb'].error models['logistic'].error
test_lem()
/print "lem" models['xgb'].error models['logistic'].error

8640 7680 960
raw (638, 257) (803, 143)
lem (676, 233) (960, 0)


In [ ]:
sqi = model.predicted.reshape(sample_indices_test.shape)


In [131]:
from sklearn.manifold import TSNE
tsne = TSNE(3, perplexity=30)
t0 = time()
fitted = tsne.fit_transform(XX0.reshape((XX0.shape[0], XX0.shape[1] * XX0.shape[2])))
t1 = time()
t1 - t0

234.04964756965637

### Bonus t-SNE

In [132]:
trace = go.Scatter3d(x=fitted[:, 0], y=fitted[:, 1], z=fitted[:, 2], mode='markers',
                     marker=dict(size='2', color=label_train * 1, showscale=True, colorscale='Jet', opacity=.8))
py.iplot([trace])

In [82]:
### random stuff

array([3, 7])

In [69]:
import plotly 
plotly.tools.set_credentials_file(username='michaely', api_key='G6eLVa3Tt5aFJQQMqvgg')

In [78]:
class samp40nnn800(Ob()):
    sample_windows0 = sample_windows0
    sample_indices0 = sample_windows0
    sample_windows = sample_windows
    sample_indices = sample_windows
    sample_windows_test = sample_windows_test
    sample_indices_test = sample_windows_test